In [1]:
import pypsa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\networkclustering.py:16: UserWarning: The namespace `pypsa.networkclustering` is deprecated and will be removed in PyPSA v0.24. Please use `pypsa.clustering.spatial instead`. 
  warnings.warn(


In [2]:
network = 'C:/Users/noraky/Documents/Test_4/pypsa-earth/networks/elec_s_27_ec_lcopt_Co2L-1H.nc'
n_27 = pypsa.Network(network)

c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with

In [4]:
n = pypsa.Network()
years = [2020,2021,2022,2023,2024,2025]
freq = "24"

snapshots = pd.DatetimeIndex([])
for year in years:
    period = pd.date_range(
        start="{}-01-01 00:00".format(year),
        freq="{}H".format(freq),
        periods=8760 / float(freq),
    )
    snapshots = snapshots.append(period)

# convert to multiindex and assign to network
n.snapshots = pd.MultiIndex.from_arrays([snapshots.year, snapshots])
n.investment_periods = years

n.snapshot_weightings

c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with

objective  stores  generators
period timestep                                 
2020   2020-01-01        1.0     1.0         1.0
       2020-01-02        1.0     1.0         1.0
       2020-01-03        1.0     1.0         1.0
       2020-01-04        1.0     1.0         1.0
       2020-01-05        1.0     1.0         1.0
...                      ...     ...         ...
2025   2025-12-27        1.0     1.0         1.0
       2025-12-28        1.0     1.0         1.0
       2025-12-29        1.0     1.0         1.0
       2025-12-30        1.0     1.0         1.0
       2025-12-31        1.0     1.0         1.0

[2190 rows x 3 columns]

In [5]:
n.investment_period_weightings["years"] = list(np.diff(years)) + [10]

r = 0.01
T = 0
for period, nyears in n.investment_period_weightings.years.items():
    discounts = [(1 / (1 + r) ** t) for t in range(T, T + nyears)]
    n.investment_period_weightings.at[period, "objective"] = sum(discounts)
    T += nyears
n.investment_period_weightings

,objective,years
2020,1.000000,1
2021,0.990099,1
2022,0.980296,1
2023,0.970590,1
2024,0.960980,1
2025,9.101737,10


In [7]:
n.lines = n_27.lines
n.generators = n_27.generators
n.storage_units = n_27.storage_units
n.loads_t = n_27.loads_t